In [7]:
import pandas as pd
mainTable = pd.read_csv("./mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz", compression='gzip', usecols=['SUBJECT_ID', "CHARTTIME", "ITEMID"])
print(mainTable.head(5))

   SUBJECT_ID  ITEMID            CHARTTIME
0          36  223834  2134-05-12 12:00:00
1          36  223835  2134-05-12 12:00:00
2          36  224328  2134-05-12 12:00:00
3          36  224329  2134-05-12 12:00:00
4          36  224330  2134-05-12 12:00:00


In [8]:
D_ITEMS = pd.read_csv('./mimic-iii-clinical-database-1.4/D_ITEMS.csv.gz', compression='gzip')
print(D_ITEMS.head(5))
print()
print()
print(D_ITEMS[D_ITEMS['ITEMID'] == 762])
itemsD = dict(zip(D_ITEMS['ITEMID'], D_ITEMS['LABEL']))
print(itemsD[762])
print(len(itemsD))

   ROW_ID  ITEMID                                        LABEL ABBREVIATION  \
0     457     497  Patient controlled analgesia (PCA) [Inject]          NaN   
1     458     498                            PCA Lockout (Min)          NaN   
2     459     499                               PCA Medication          NaN   
3     460     500                               PCA Total Dose          NaN   
4     461     501                           PCV Exh Vt (Obser)          NaN   

  DBSOURCE      LINKSTO CATEGORY UNITNAME PARAM_TYPE  CONCEPTID  
0  carevue  chartevents      NaN      NaN        NaN        NaN  
1  carevue  chartevents      NaN      NaN        NaN        NaN  
2  carevue  chartevents      NaN      NaN        NaN        NaN  
3  carevue  chartevents      NaN      NaN        NaN        NaN  
4  carevue  chartevents      NaN      NaN        NaN        NaN  


      ROW_ID  ITEMID     LABEL ABBREVIATION DBSOURCE      LINKSTO CATEGORY  \
1506     687     762  Admit Wt          NaN  care

In [9]:
PATIENTS = pd.read_csv('./mimic-iii-clinical-database-1.4/PATIENTS.csv.gz', compression='gzip')
print(PATIENTS.head(5))
print(PATIENTS[PATIENTS['SUBJECT_ID'] == 2])

patientD = dict(zip(PATIENTS["SUBJECT_ID"], PATIENTS["DOB"]))
print(patientD[2])

   ROW_ID  SUBJECT_ID GENDER                  DOB                  DOD  \
0     234         249      F  2075-03-13 00:00:00                  NaN   
1     235         250      F  2164-12-27 00:00:00  2188-11-22 00:00:00   
2     236         251      M  2090-03-15 00:00:00                  NaN   
3     237         252      M  2078-03-06 00:00:00                  NaN   
4     238         253      F  2089-11-26 00:00:00                  NaN   

              DOD_HOSP DOD_SSN  EXPIRE_FLAG  
0                  NaN     NaN            0  
1  2188-11-22 00:00:00     NaN            1  
2                  NaN     NaN            0  
3                  NaN     NaN            0  
4                  NaN     NaN            0  
     ROW_ID  SUBJECT_ID GENDER                  DOB  DOD DOD_HOSP DOD_SSN  \
222       1           2      M  2138-07-17 00:00:00  NaN      NaN     NaN   

     EXPIRE_FLAG  
222            0  
2138-07-17 00:00:00
